# HashiCorp Vault Demo for Rotating Linux user passwords
This demo shows we can utilize a script to automate the process of rotating passwords for a defined list of linux servers.  All rotated passwords are stored securely in HashiCorp Vault KV v2 engine.  Each rotated password is stored as a different version.

## Setup of the Demo

This setup is tested on MacOS and is meant to simulate a distributed setup.  The components used in this demo are:
- Vault Enterprise installed on docker (to simulate an external Vault)
- You have the Vault CLI installed

This assumes your Vault server is installed using docker and already running on http://127.0.0.1:8200
and you have set your VAULT_ADDR and VAULT_TOKEN variables.

Note: You will need Vault to be installed with an ADP Transform license add-on.

## Requirements to Run This Demo
You will need Visual Studio Code to be installed with the Jupyter plugin.  To run this notebook in VS Code, chose the Jupyter kernel and then Bash.
- To run the current cell, use Ctrl + Enter.
- To run the current cell and advance to the next, use Shift+Enter.

# Setup Pre-requisites (One-time)

Assumes you have docker installed and brew installed

- https://docs.docker.com/desktop/install/mac-install/
- https://brew.sh/

In [ ]:
# sshpass is used in the rotation script to login via ssh with password
brew install sshpass

In [ ]:
# jq is used in the script to parse one of the json output
brew install jq

# Setting up HashiCorp Vault

In [ ]:
# Optional.  The following are some sample commands for running Vault Enterprise in docker.
export VAULT_PORT=8200
export VAULT_ADDR="http://127.0.0.1:${VAULT_PORT}"
export VAULT_TOKEN="root"
# Change the path to your license file
export VAULT_LICENSE=$(cat $HOME/vault-enterprise/vault_local/data/vault.hclic)
docker run -d --rm --name vault-enterprise --cap-add=IPC_LOCK \
-e "VAULT_DEV_ROOT_TOKEN_ID=${VAULT_TOKEN}" \
-e "VAULT_DEV_LISTEN_ADDRESS=:${VAULT_PORT}" \
-e "VAULT_LICENSE=${VAULT_LICENSE}" \
-p ${VAULT_PORT}:${VAULT_PORT} hashicorp/vault-enterprise:latest

In [ ]:
# Testing with ubuntu image with sshd enabled
# https://hub.docker.com/r/takeyamajp/ubuntu-sshd
export INT_SSH_PORT=22
export EXT_SSH_PORT=8222
docker run -it -d --rm --name ubuntu \
-e ROOT_PASSWORD=root \
-p ${EXT_SSH_PORT}:${INT_SSH_PORT} takeyamajp/ubuntu-sshd:latest



In [ ]:
# Verify that Vault and the Ubuntu containers are running
docker ps

In [ ]:
# Show that we are rotating the servers that we specified.  One is valid "localhost" and the other is not "test".
cat hostlist.txt

In [ ]:
# Rotate passwords
./rotate.sh

In [ ]:
# Check that the password has been rotated
export SSHPASS=$(vault kv get -field=admin-password -mount=secret localhost)
echo "Rotated password for localhost: $SSHPASS"

In [ ]:
# Verify that you get an error with an invalid password.  You should see a permission denied error.
sshpass -p "wrong_password" ssh -o StrictHostKeyChecking=no -p $EXT_SSH_PORT root@localhost ls

In [ ]:
# Verify that I can SSH and run a command using the new password
# Sould have no errors
sshpass -e ssh -o StrictHostKeyChecking=no -p $EXT_SSH_PORT root@localhost ls

# Cleanup

In [ ]:
# Cleanup
# stop Ubuntu container
docker stop ubuntu

# Stop Vault container
docker stop vault-enterprise

# Other Useful Commands and Examples

In [ ]:
# Generate a string of random bytes
# https://developer.hashicorp.com/vault/api-docs/system/tools#generate-random-bytes
# e.g. AES 256 is 256 bits or 32 bytes
# Format can be base64 or hex
vault write sys/tools/random/32 format=base64
echo
vault write sys/tools/random/32 format=hex
echo 
vault write -field=random_bytes sys/tools/random/32 format=base64
echo
vault write -field=random_bytes sys/tools/random/32 format=hex

In [ ]:
# Example of generating a random 256-bit key (32 bytes) and storing it as a static secret in the KV engine
# Variable for my secret path
MOUNTPATH=secret
KEYNAME=demo-app-key-1
VALUENAME=value
# Generates a 256-bit key in base64 and stores it in the secret path with the name "value"
vault kv put $MOUNTPATH/$KEYNAME $VALUENAME="$(vault write -field=random_bytes sys/tools/random/32 format=base64)"
echo
# Application can read back key value with the proper permission policy.
vault kv get -mount=$MOUNTPATH -field $VALUENAME $KEYNAME

In [ ]:
# Demo of -output-policy flag
vault write -output-policy sys/tools/random/32 format=base64

In [ ]:
# View all local users
docker exec -it ubuntu cut -d: -f1 /etc/passwd